In [1]:
import talib as ta
import pandas as pd
import numpy as np

In [2]:
# read the data from the CSV files
train_data = pd.read_csv('train_data_50.csv') 

# change date into datetime objects
train_data['Date'] = pd.to_datetime(train_data['Date'])

# set indexes 
train_data.set_index(["Ticker", "Date"], inplace=True)

# sort data by tickers
tickers = sorted(train_data.index.get_level_values('Ticker').unique())

In [3]:
# array of all stocks and close prices
open_prices = []

for ticker in tickers:
    stock_close_data = train_data.loc[ticker]["Open"]
    open_prices.append(stock_close_data.values)

open_prices = np.stack(open_prices)
print(open_prices.shape)
print(open_prices)

# array for final return - trades
trades = np.zeros_like(open_prices)
trades

(50, 1452)
[[17.04294995 17.25096871 16.96448663 ... 67.35797313 68.06231506
  68.62432851]
 [12.20377331 12.24209241 12.04319784 ... 27.93657064 28.10079496
  28.43107009]
 [27.76139563 28.32888582 28.12086706 ... 45.28424827 45.45394932
  45.20031238]
 ...
 [12.73111928 12.80593376 12.77856257 ... 40.5016405  40.69141088
  40.9340999 ]
 [ 8.02149948  8.0908393   8.10908629 ... 45.21855798 46.12362377
  46.14186937]
 [20.12126341 20.38584965 20.38584965 ... 57.08475709 57.0592138
  57.54641278]]


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [4]:
# calculate golden cross value using TA-lib
# golden cross is basically the same as the SMA example given

for stock in range(len(open_prices)): 
    fast_sma = ta.SMA(open_prices[stock], timeperiod=5)
    slow_sma = ta.SMA(open_prices[stock], timeperiod=40)

    for day in range(1, len(open_prices[0])-1):
        
        # Buy: fast SMA crosses above slow SMA
        if fast_sma[day] > slow_sma[day] and fast_sma[day-1] <= slow_sma[day-1]:
            # we are trading the next day's open price
            trades[stock][day+1] = 1
        
        # Sell/short: fast SMA crosses below slow SMA
        elif fast_sma[day] < slow_sma[day] and fast_sma[day-1] >= slow_sma[day-1]:
            # we are trading the next day's open price
            trades[stock][day+1] = -1
        # else do nothing
        else:
            trades[stock][day+1] = 0

In [5]:
from eval_algo import eval_actions

portfolio_value, sharpe_ratio = eval_actions(trades, open_prices, cash=25000, verbose=True)
print(f"\nPortfolio value: {portfolio_value}")
print(f"Sharpe ratio: {sharpe_ratio}")

final portfolio value: 25459.82057448075
cash: 24033.604228822827
positions: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -19.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0]
short position info: defaultdict(<class 'list'>, {29: []})
short value: 0

Portfolio value: [25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000.306737251023, 24999.293467514344, 24998.293060952714, 24998.393785888442, 24998.393785888442, 24998.393785888442, 24998.284304290355, 24999.240460123765, 24999.2021416993, 24999.74709588395, 24999.505866063322, 24999.000781143364, 24999.805821208, 24998.376421654957, 24998.644